In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Libraries

In [12]:
from torchvision.transforms import Compose, ToTensor
from torchsummary import summary
import pickle
import pandas as pd
import plotly.express as px
from tqdm import tqdm

In [13]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training import EWC
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import accuracy_metrics, forgetting_metrics, bwt_metrics, timing_metrics, cpu_usage_metrics, ram_usage_metrics

from torch.nn import CrossEntropyLoss
from torch.optim import SGD

## Custom Libraries

In [14]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.plugins import WeightStoragePlugin

# Dataset and definitions

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [15]:
scenario = PermutedMNIST(10, seed=1234, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [16]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

## Evaluation metrics definition

In [17]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True, trained_experience=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(experience=True, stream=True),
    ram_usage_metrics(experience=True, stream=True),
    loggers=[InteractiveLogger()]
)

## Plugin defitinitions

In [18]:
model_plugins = [WeightStoragePlugin()]

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [19]:
# model = MLP(n_classes=scenario.n_classes, n_channels=1, width=28, height=28)
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100)
optimizer = SGD(model.parameters(), lr=1e-3)
criterion = CrossEntropyLoss()
strategy = EWC(
    model,
    optimizer,
    criterion,
    ewc_lambda=1.0,
    train_epochs=5,
    train_mb_size=128,
    plugins=model_plugins,
    evaluator=eval_plugin,
    eval_mb_size=128,
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [20]:
results = list()

for experience in tqdm(train_stream):
    strategy.train(experience)

    # eval on the whole train stream
    metrics = strategy.eval(train_stream)
    results.append(metrics)

    # eval on test
    metrics = strategy.eval(test_stream)
    results.append(metrics)

  0%|          | 0/10 [00:00<?, ?it/s]

-- >> Start of training phase << --
100%|██████████| 469/469 [00:03<00:00, 126.00it/s]
Epoch 0 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0002
	Time_Epoch/train_phase/train_stream/Task000 = 3.7224
100%|██████████| 469/469 [00:03<00:00, 130.95it/s]
Epoch 1 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 3.5817
100%|██████████| 469/469 [00:03<00:00, 131.75it/s]
Epoch 2 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 3.5600
100%|██████████| 469/469 [00:03<00:00, 129.81it/s]
Epoch 3 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 3.6132
100%|██████████| 469/469 [00:03<00:00, 130.86it/s]
Epoch 4 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 3.5843
-- >> End of training phase << --
-- >> Start of eval phase

 10%|█         | 1/10 [01:01<09:16, 61.88s/it]

-- >> Start of training phase << --
100%|██████████| 469/469 [00:03<00:00, 121.36it/s]
Epoch 0 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 3.8653
100%|██████████| 469/469 [00:03<00:00, 123.72it/s]
Epoch 1 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 3.7912
100%|██████████| 469/469 [00:03<00:00, 123.21it/s]
Epoch 2 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 3.8067
100%|██████████| 469/469 [00:03<00:00, 121.80it/s]
Epoch 3 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 3.8509
100%|██████████| 469/469 [00:03<00:00, 121.00it/s]
Epoch 4 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 3.8760
-- >> End of training phase << --
-- >> Start of eval phase

 20%|██        | 2/10 [02:06<08:30, 63.76s/it]

...

-- >> End of eval phase << --
	Accuracy_On_Trained_Experiences/eval_phase/test_stream/Task000 = 0.7315
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 105.9690
	MaxRAMUsage_Stream/eval_phase/test_stream/Task000 = 161.5000
	StreamBWT/eval_phase/test_stream = -0.0703
	StreamForgetting/eval_phase/test_stream = 0.0703
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.6093
-- >> Start of training phase << --
100%|██████████| 469/469 [00:05<00:00, 87.80it/s]
Epoch 0 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 5.3423
100%|██████████| 469/469 [00:05<00:00, 88.11it/s]
Epoch 1 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 5.3233
100%|██████████| 469/469 [00:05<00:00, 87.03it/s]
Epoch 2 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 5.3891
100%|██████████| 469/469 [00:05<00:00, 87.60it/s]

 90%|█████████ | 9/10 [10:13<01:10, 70.41s/it]

-- >> End of eval phase << --
	Accuracy_On_Trained_Experiences/eval_phase/test_stream/Task000 = 0.7308
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 106.0198
	MaxRAMUsage_Stream/eval_phase/test_stream/Task000 = 162.8125
	StreamBWT/eval_phase/test_stream = -0.0741
	StreamForgetting/eval_phase/test_stream = 0.0741
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.6706
-- >> Start of training phase << --
100%|██████████| 469/469 [00:05<00:00, 84.28it/s]
Epoch 0 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 5.5653
100%|██████████| 469/469 [00:05<00:00, 84.54it/s]
Epoch 1 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 5.5481
100%|██████████| 469/469 [00:05<00:00, 83.99it/s]
Epoch 2 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 5.5841
100%|██████████| 469/469 [00:05<00:00, 84.19it/s]

100%|██████████| 10/10 [11:26<00:00, 68.67s/it]

-- >> End of eval phase << --
	Accuracy_On_Trained_Experiences/eval_phase/test_stream/Task000 = 0.7283
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 105.6563
	MaxRAMUsage_Stream/eval_phase/test_stream/Task000 = 147.6406
	StreamBWT/eval_phase/test_stream = -0.0783
	StreamForgetting/eval_phase/test_stream = 0.0783
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.7283


# Get metrics

## Training Accuracies

In [21]:
training_accuracies: dict[int, list[float]] = dict()

for i in range(10):
    training_accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/train_stream/Task000/Exp00{i}"
    ][1]

training_accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/train_stream/Task000"
][1]

## Evaluation Accuracies

In [22]:
accuracies: dict[int, list[float]] = dict()

for i in range(10):
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp00{i}"
    ][1]

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1]

## Forgetting measure

In [23]:
forgetting_measures: dict[int, list[float]] = dict()

for i in range(9):
    forgetting_measures[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceForgetting/eval_phase/test_stream/Task000/Exp00{i}"
    ][1]
forgetting_measures["Overall"] = eval_plugin.get_all_metrics()[
    "StreamForgetting/eval_phase/test_stream"
][1]

## Backward Transfer

In [24]:
bwts: dict[int, list[float]] = dict()

for i in range(9):
    bwts[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceBWT/eval_phase/test_stream/Task000/Exp00{i}"
    ][1]
bwts["Overall"] = eval_plugin.get_all_metrics()[
    "StreamBWT/eval_phase/test_stream"
][1]

# Plotting metrics

## Traning accuracies

In [25]:
train_df = pd.DataFrame(training_accuracies)
train_df.index = range(10)

fig = px.line(train_df, x=train_df.index, y=train_df.columns, range_y=[0, 1], title="Training Accuracy vs Task")
fig.show()

## Evaluation accuracies per experience

In [26]:
acc_df = pd.DataFrame(accuracies)
acc_df.index = range(10)

fig = px.line(acc_df, x=acc_df.index, y=acc_df.columns, range_y=[0, 1], title="Test Accuracy vs Task")
fig.show()

## Forgetting measure / BWT

In [27]:
from copy import deepcopy
# transform forgetting_measures dict into df
# but first, we need to make sure that all lists have the same length
max_len = max(map(len, forgetting_measures.values()))
forgetting_measures_tmp = deepcopy(forgetting_measures)
forgetting_measures_tmp = {k: [None] * (max_len - len(v)) + v for k, v in forgetting_measures_tmp.items()}

forgetting_df = pd.DataFrame(forgetting_measures_tmp)
forgetting_df.index = range(10)

fig = px.line(forgetting_df, x=forgetting_df.index, y=forgetting_df.columns, title="Forgetting vs Task")
fig.show()

# Store metrics

In [28]:
pickle.dump(eval_plugin.get_all_metrics(), open(SAVED_METRICS_PATH / "pmnist" / "ewc.pkl", "wb"))

# Store weights

In [29]:
pickle.dump(model_plugins[0].weights, open(SAVED_METRICS_PATH / "pmnist" / "ewc_weights.pkl", "wb"))